<a href="https://colab.research.google.com/github/diogocristovao/SPBD_tp1/blob/main/spbd_tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install PySpark
!pip install pyspark findspark --quiet

In [2]:
#@title Download the dataset

!wget -q -O energy-readings.csv https://raw.githubusercontent.com/smduarte/spbd-2425/refs/heads/main/docs/labs/projs/energy-readings.csv
!head -10 energy-readings.csv

date;sensor;energy
2024-02-01 00:00:00;D;2615.0
2024-02-01 00:00:18;C;1098.8
2024-02-01 00:00:25;A;650.5
2024-02-01 00:00:33;J;966.7
2024-02-01 00:00:42;H;2145.4
2024-02-01 00:00:54;E;1874.0
2024-02-01 00:01:52;K;841.2
2024-02-01 00:02:00;E;1874.1
2024-02-01 00:02:20;I;927.2


In [15]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
						.appName('energy').getOrCreate()

sc = spark.sparkContext
try :
    readings = spark.read.csv('energy-readings.csv',
                             sep =';', header=True, inferSchema=True)

    readings.printSchema()


    readings.show(11)
except Exception as err:
    print(err)

root
 |-- date: timestamp (nullable = true)
 |-- sensor: string (nullable = true)
 |-- energy: double (nullable = true)

+-------------------+------+------+
|               date|sensor|energy|
+-------------------+------+------+
|2024-02-01 00:00:00|     D|2615.0|
|2024-02-01 00:00:18|     C|1098.8|
|2024-02-01 00:00:25|     A| 650.5|
|2024-02-01 00:00:33|     J| 966.7|
|2024-02-01 00:00:42|     H|2145.4|
|2024-02-01 00:00:54|     E|1874.0|
|2024-02-01 00:01:52|     K| 841.2|
|2024-02-01 00:02:00|     E|1874.1|
|2024-02-01 00:02:20|     I| 927.2|
|2024-02-01 00:02:36|     K| 841.3|
|2024-02-01 00:03:24|     G| 833.7|
+-------------------+------+------+
only showing top 11 rows



In [30]:
try:
    #loading the csv file
    readings = spark.read.csv('energy-readings.csv',
                              sep=';', header=True, inferSchema=True)

    #display the data schema and type
    readings.printSchema()

    # filter only the data that refers to february of 2024, as required
    readings_february = readings.filter((year("date") == 2024) & (month("date") == 2))

    # grouping the data by sensor and computing the max and min value of energy for each sensor ( we will subtract the min to the max to obtain the energy consumed by each sensor during february and use those values in question 1)
    energy_per_sensor = readings_february.groupBy("sensor").agg(
        max("energy").alias("max_energy"),
        min("energy").alias("min_energy")
    ).orderBy("sensor")

    # display the max and min energy for each sensor
    energy_per_sensor.show()

except Exception as err:
   print(err)


root
 |-- date: timestamp (nullable = true)
 |-- sensor: string (nullable = true)
 |-- energy: double (nullable = true)

+------+----------+----------+
|sensor|max_energy|min_energy|
+------+----------+----------+
|     A|    816.88|     650.5|
|     B|    757.31|     627.5|
|     C|   1356.02|    1098.8|
|     D|    3102.4|    2615.0|
|     E|   2322.76|    1874.0|
|     F|    908.41|     748.0|
|     G|   1002.17|     833.7|
|     H|    2625.0|    2145.4|
|     I|   1278.61|     927.2|
|     J|   1197.55|     966.7|
|     K|    1067.7|     841.2|
+------+----------+----------+



In [31]:
#@title Question 1

try:


    # subtracting the min energy to the max energy , in february, for each sensor
    total_energy = readings_february.groupBy("sensor").agg(
        round((max("energy") - min("energy")), 2).alias("total_energy_consumed (Kwh)")
    ).orderBy("sensor")

    # sums the energy consumed by each sensor
    total_energy_sum = total_energy.agg(round(sum("total_energy_consumed (Kwh)"), 2).alias("total_energy_all_sensors")).collect()[0]["total_energy_all_sensors"]

    total_energy.show()

    print("Total energy consumed by all sensors:", total_energy_sum, "(Kwh)")


except Exception as err:
   print(err)

+------+---------------------------+
|sensor|total_energy_consumed (Kwh)|
+------+---------------------------+
|     A|                     166.38|
|     B|                     129.81|
|     C|                     257.22|
|     D|                      487.4|
|     E|                     448.76|
|     F|                     160.41|
|     G|                     168.47|
|     H|                      479.6|
|     I|                     351.41|
|     J|                     230.85|
|     K|                      226.5|
+------+---------------------------+

Total energy consumed by all sensors: 3106.81 (Kwh)


In [39]:
#@title Question 2
try:

    #transforming the "date" column from timestamp to DateType
    readings_february = readings_february.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))

    # using window functions to get the last reading of the day for each sensor
    window_spec = Window.partitionBy("sensor", "date").orderBy(desc("date"))

    #creates a temporary column "row_number" that identifies the last reading of the day for each sensor, that reading will have "row_number"=1
    daily_last_reading = readings_february.withColumn("row_number", row_number().over(window_spec)) \
                                 .filter(col("row_number") == 1) \
                                 .drop("row_number")

    #sum of the last readings for the group of sensors
    daily_running_total = daily_last_reading.groupBy("date").agg(
        round(sum("energy"), 2).alias("running_total_energy (Kwh)")
    ).orderBy("date")


    daily_running_total.show(truncate=False)

except Exception as err:
   print(err)

+----------+--------------------------+
|date      |running_total_energy (Kwh)|
+----------+--------------------------+
|2024-02-01|13328.0                   |
|2024-02-02|13448.3                   |
|2024-02-09|14377.2                   |
|2024-02-10|14433.5                   |
|2024-02-11|14547.6                   |
|2024-02-12|14665.6                   |
|2024-02-13|14776.3                   |
|2024-02-14|14889.3                   |
|2024-02-15|14982.4                   |
|2024-02-16|15063.8                   |
|2024-02-18|15293.6                   |
|2024-02-19|15351.6                   |
|2024-02-20|15431.4                   |
|2024-02-21|15515.4                   |
|2024-02-22|15598.5                   |
|2024-02-23|15675.4                   |
|2024-02-24|15839.8                   |
|2024-02-25|15903.37                  |
|2024-02-26|16003.19                  |
|2024-02-27|16095.89                  |
+----------+--------------------------+
only showing top 20 rows





For each sensor, separately:
Compute the total energy consumed and the average energy consumption per day.


In [33]:
#@title Question 3
try:

    # energy consumed by each sensor ( februrary), once again
    total_energy = readings_february.groupBy("sensor").agg(
        round((max("energy") - min("energy")),2).alias("total_energy_consumed (Kwh)")
    )

    #divides that total energy by the number of days in february 2024, which were 29
    total_energy = total_energy.withColumn(
        "average_daily_energy_consumed (Kwh)", round(col("total_energy_consumed (Kwh)") / 29, 2)
    )


    total_energy.select("sensor", "total_energy_consumed (Kwh)", "average_daily_energy_consumed (Kwh)") \
                .orderBy("sensor") \
                .show()

except Exception as err:
    print("Erro:", err)


+------+---------------------------+-----------------------------------+
|sensor|total_energy_consumed (Kwh)|average_daily_energy_consumed (Kwh)|
+------+---------------------------+-----------------------------------+
|     A|                     166.38|                               5.74|
|     B|                     129.81|                               4.48|
|     C|                     257.22|                               8.87|
|     D|                      487.4|                              16.81|
|     E|                     448.76|                              15.47|
|     F|                     160.41|                               5.53|
|     G|                     168.47|                               5.81|
|     H|                      479.6|                              16.54|
|     I|                     351.41|                              12.12|
|     J|                     230.85|                               7.96|
|     K|                      226.5|               

For each sensor, separately:

Compute the day of the month with minimum and maximum energy consumption.

In [34]:
#@title Question 4
try:


    # groups the readings by sensor and day for each sensor
    daily_consumption = readings_february.groupBy("sensor", "date").agg(
        first("energy").alias("first_reading"),
        last("energy").alias("last_reading")
    )
    # computes the daily energy consumption for each sensor
    daily_consumption = daily_consumption.withColumn(
        "daily_energy_consumption", col("last_reading") - col("first_reading")
    )

    # identifying the day in february with the min energy consumption
    min_consumption_day = daily_consumption.groupBy("sensor").agg(
        round(min("daily_energy_consumption"), 3).alias("min_daily_energy"),
        first("date").alias("day_min_consumption")  # Ajustar para mostrar o dia correto
    )
    #same reasoning for the max value
    max_consumption_day = daily_consumption.groupBy("sensor").agg(
        round(max("daily_energy_consumption"), 3).alias("max_daily_energy"),
        last("date").alias("day_max_consumption")  # Ajustar para mostrar o dia correto
    )


    result = min_consumption_day.join(max_consumption_day, on="sensor")


    result.select("sensor", "day_min_consumption", "min_daily_energy", "day_max_consumption", "max_daily_energy") \
          .orderBy("sensor") \
          .show()

except Exception as err:
    print("Erro:", err)


+------+-------------------+----------------+-------------------+----------------+
|sensor|day_min_consumption|min_daily_energy|day_max_consumption|max_daily_energy|
+------+-------------------+----------------+-------------------+----------------+
|     A|         2024-02-28|            0.77|         2024-02-23|             8.1|
|     B|         2024-02-26|             0.1|         2024-02-22|             9.9|
|     C|         2024-02-23|             1.6|         2024-02-12|            14.0|
|     D|         2024-02-11|             5.7|         2024-02-21|            26.4|
|     E|         2024-02-16|             4.7|         2024-02-13|            20.6|
|     F|         2024-02-27|             0.8|         2024-02-09|           12.87|
|     G|         2024-02-22|             0.7|         2024-02-18|             9.3|
|     H|         2024-02-15|             2.1|         2024-02-20|           26.11|
|     I|         2024-02-02|             0.5|         2024-02-23|           20.66|
|   

In [35]:
#@title Question 5

from pyspark.sql.window import Window
from datetime import datetime, timedelta

#for questions 5 and 6, we need the "date" column in its original format, timestamp
readings_february = readings_february.withColumn("date", to_timestamp(col("date")))



target_hours = []
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 3, 1)
current_date = start_date

# Gera um timestamp para cada hora de fevereiro
while current_date < end_date:
    target_hours.append((current_date,))
    current_date += timedelta(hours=1)

# Converte para DataFrame de horários alvo
target_hours_df = spark.createDataFrame(target_hours, ["target_timestamp"])

# Passo 2: CrossJoin para garantir que cada sensor tenha todos os horários alvo
sensors_df = readings.select("sensor").distinct()
target_hours_df = target_hours_df.crossJoin(sensors_df)

# Passo 3: Juntar readings com target_hours_df para cada sensor
# Adiciona colunas 'prev_energy', 'next_energy', 'prev_date' e 'next_date' para cada sensor e hora alvo
window_spec = Window.partitionBy("sensor").orderBy("date")
readings = readings.withColumn("prev_energy", lag("energy").over(window_spec)) \
                   .withColumn("next_energy", lead("energy").over(window_spec)) \
                   .withColumn("prev_date", lag("date").over(window_spec)) \
                   .withColumn("next_date", lead("date").over(window_spec))

# Junta com target_hours_df
readings_with_targets = target_hours_df.join(readings, "sensor")

# Filtra para manter apenas as leituras anteriores e posteriores de cada hora alvo
readings_with_targets = readings_with_targets.filter(
    (col("prev_date").isNotNull()) &
    (col("next_date").isNotNull()) &
    (col("prev_date") <= col("target_timestamp")) &
    (col("next_date") >= col("target_timestamp"))
)

# Calcula o valor interpolado
readings_with_targets = readings_with_targets.withColumn(
    "interpolated_energy",
    when(
        col("prev_energy").isNotNull() & col("next_energy").isNotNull(),
        # Interpolação linear
        round(col("prev_energy") +
        (col("next_energy") - col("prev_energy")) *
        ((unix_timestamp("target_timestamp") - unix_timestamp("prev_date")) /
         (unix_timestamp("next_date") - unix_timestamp("prev_date")))
    ,1)).otherwise(None)
)

# Usa uma janela para cada sensor e target_timestamp, pegando apenas o primeiro valor interpolado
window_spec_unique = Window.partitionBy("sensor", "target_timestamp").orderBy("date")
readings_with_targets = readings_with_targets.withColumn(
    "interpolated_energy_unique", first("interpolated_energy").over(window_spec_unique)
)

# Seleciona as colunas finais e remove duplicatas
readings_with_targets = readings_with_targets.select("sensor", "target_timestamp", "interpolated_energy_unique") \
                                             .withColumnRenamed("interpolated_energy_unique", "interpolated_energy") \
                                             .distinct() \
                                             .orderBy("sensor", "target_timestamp")

# Exibe os resultados
#readings_with_targets.show(n=300)
# Filtra para mostrar apenas os resultados do sensor específico, por exemplo, "C"
readings_with_targets.filter(col("sensor") == "C") \
                     .orderBy("target_timestamp") \
                     .show(n=300)


+------+-------------------+-------------------+
|sensor|   target_timestamp|interpolated_energy|
+------+-------------------+-------------------+
|     C|2024-02-01 01:00:00|             1098.8|
|     C|2024-02-01 02:00:00|             1098.8|
|     C|2024-02-01 03:00:00|             1098.9|
|     C|2024-02-01 04:00:00|             1098.9|
|     C|2024-02-01 05:00:00|             1098.9|
|     C|2024-02-01 06:00:00|             1099.0|
|     C|2024-02-01 07:00:00|             1099.1|
|     C|2024-02-01 08:00:00|             1099.7|
|     C|2024-02-01 09:00:00|             1099.9|
|     C|2024-02-01 10:00:00|             1100.0|
|     C|2024-02-01 11:00:00|             1100.0|
|     C|2024-02-01 12:00:00|             1100.0|
|     C|2024-02-01 13:00:00|             1100.0|
|     C|2024-02-01 14:00:00|             1100.3|
|     C|2024-02-01 15:00:00|             1100.9|
|     C|2024-02-01 16:00:00|             1102.2|
|     C|2024-02-01 17:00:00|             1102.4|
|     C|2024-02-01 1

In [36]:
#@title Question 6

# Define uma janela para calcular o consumo de energia por hora para cada sensor
window_spec_hourly = Window.partitionBy("sensor").orderBy("target_timestamp")

# Calcula o consumo de energia por hora (diferença entre leituras consecutivas)
sensor_hourly_consumption = readings_with_targets.withColumn(
    "hourly_energy_consumed",
    col("interpolated_energy") - lag("interpolated_energy").over(window_spec_hourly)
).fillna(0, subset=["hourly_energy_consumed"])  # Substitui valores nulos pela primeira leitura como 0

# Define uma janela de ordenação para calcular o total acumulado por sensor
window_spec_cumulative = Window.partitionBy("sensor").orderBy("target_timestamp").rowsBetween(Window.unboundedPreceding, 0)

# Calcula o total acumulado de energia consumida para cada sensor ao longo do tempo
sensor_running_total_energy = sensor_hourly_consumption.withColumn(
    "running_total_energy",
    round(sum("hourly_energy_consumed").over(window_spec_cumulative), 1)
)

# Exibe apenas as colunas sensor, target_timestamp e running_total_energy
sensor_running_total_energy.select("sensor", "target_timestamp", "running_total_energy") \
                           .orderBy("sensor", "target_timestamp") \
                           .show(n=30)


+------+-------------------+--------------------+
|sensor|   target_timestamp|running_total_energy|
+------+-------------------+--------------------+
|     A|2024-02-01 01:00:00|                 0.0|
|     A|2024-02-01 02:00:00|                 0.1|
|     A|2024-02-01 03:00:00|                 0.1|
|     A|2024-02-01 04:00:00|                 0.1|
|     A|2024-02-01 05:00:00|                 0.4|
|     A|2024-02-01 06:00:00|                 0.6|
|     A|2024-02-01 07:00:00|                 0.6|
|     A|2024-02-01 08:00:00|                 0.6|
|     A|2024-02-01 09:00:00|                 0.7|
|     A|2024-02-01 10:00:00|                 0.7|
|     A|2024-02-01 11:00:00|                 1.6|
|     A|2024-02-01 12:00:00|                 1.7|
|     A|2024-02-01 13:00:00|                 2.7|
|     A|2024-02-01 14:00:00|                 3.9|
|     A|2024-02-01 15:00:00|                 4.0|
|     A|2024-02-01 16:00:00|                 4.0|
|     A|2024-02-01 17:00:00|                 4.1|
